## 02: Query Feasibility Verification

For each of the 24 benchmark queries, this notebook:
1. Shows the original SQL and what `reporting.system_*` tables it references
2. Maps each reporting table to our actual data file (parquet or gz)
3. Verifies every column the query needs exists in our data
4. Notes any aggregation/transformation needed to bridge raw → reporting schema

Data inventory: ~20.7 GiB across parquet tables + gzipped text files from `dca_update_dec_2024`.

24/24 queries covered:
- 21 queries with multi-guid population data (thousands of clients)
- 3 queries (`ranked_process_classifications`, `top_10_processes_per_user_id_ranked_by_total_power_consumption`, `top_20_most_power_consuming_processes_by_avg_power_consumed`) use single-guid stub data from `mods_sleepstudy_power_estimation_data_13wks` (10K rows, 1 guid). These queries don't reference `guid`, they aggregate by `user_id`/`app_id` only, so the data is sufficient for the DP benchmark comparison. See Queries 22-24 for details.

In [11]:
from pathlib import Path

import duckdb
from IPython.display import display, Markdown

DATA = Path("../data/raw")
QUERIES = Path("../docs/queries")
con = duckdb.connect()


def check_columns(source_path: str, needed_cols: list[str], *, read_func: str = "auto", ignore_errors: bool = False) -> None:
    if read_func == "auto":
        read_func = "read_parquet" if source_path.endswith(".parquet") or "*" in source_path else "read_csv"

    opts = ", auto_detect=true, ignore_errors=true" if (read_func == "read_csv" and ignore_errors) else (", auto_detect=true" if read_func == "read_csv" else "")
    
    schema = con.execute(f"DESCRIBE SELECT * FROM {read_func}('{source_path}'{opts})").df()
    stats = con.execute(f"SELECT COUNT(*) as n, COUNT(DISTINCT guid) as g FROM {read_func}('{source_path}'{opts})").fetchone()
    
    available = set(schema["column_name"].tolist())
    lines = [f"Rows: {stats[0]:,} | Guids: {stats[1]:,} | Columns: {len(available)}"]
    
    for col in needed_cols:
        if col in available:
            lines.append(f"- ✓ `{col}`")
        else:
            lines.append(f"- ✗ `{col}` MISSING")
    
    display(Markdown("\n".join(lines)))

---
## Data sources reference

| Reporting table | Data file | Format | Notes |
|---|---|---|---|
| `system_sysinfo_unique_normalized` | `data/system_sysinfo_unique_normalized/*.parquet` | Parquet (8 files) | Direct match, anchor table |
| `system_network_consumption` | `data/os_network_consumption_v2/*.parquet` | Parquet (1/8) | Needs column rename: `input_description`→`input_desc`, `nr_samples`→`nrs` |
| `system_web_cat_usage` | `data/web_cat_usage_v2/*.parquet` | Parquet (1/8) | Direct, has `browser`, `duration_ms`, `guid` |
| `system_web_cat_pivot_duration` | `data/web_cat_pivot/*.parquet` | Parquet (8 files) | Column names use underscored full names vs query's short names |
| `system_memory_utilization` | `data/os_memsam_avail_percent/*.parquet` | Parquet (1/8) | `avg_percentage_used = 100 - average`, `nrs = sample_count`, `sysinfo_ram` via JOIN |
| `system_hw_pkg_power` | `data/hw_metric_stats/*.parquet` | Parquet (1/8) | Filter: `name LIKE '%POWER%'`, rename `mean`→power metric |
| `system_psys_rap_watts` | `data/hw_metric_stats/*.parquet` | Parquet (1/8) | Filter: `name IN ('HW::PACKAGE:RAP:WATTS:', 'HW:::PSYS_RAP:WATTS:')` |
| `system_pkg_C0` | `data/hw_metric_stats/*.parquet` | Parquet (1/8) | Filter: `name = 'HW::PACKAGE:C0_RESIDENCY:PERCENT:'` |
| `system_pkg_avg_freq_mhz` | `data/hw_metric_stats/*.parquet` | Parquet (1/8) | Filter: `name = 'HW::CORE:AVG_FREQ:MHZ:'` |
| `system_pkg_temp_centigrade` | `data/hw_metric_stats/*.parquet` | Parquet (1/8) | Filter: `name = 'HW::CORE:TEMPERATURE:CENTIGRADE:'` |
| `system_cpu_metadata` | `data/system_cpu_metadata.txt000.gz` | Gzipped CSV | Direct match |
| `system_os_codename_history` | `data/system_os_codename_history.txt000.gz` | Gzipped CSV | Direct match |
| `system_on_off_suspend_time_day` | `data/guids_on_off_suspend_time_day.txt000.gz` | Gzipped CSV | Direct match |
| `system_mods_top_blocker_hist` | `data/mods_sleepstudy_top_blocker_hist.txt000.gz` | Gzipped CSV | Column rename: `active_time_ms` (raw has this already), `dt` derived from `dt_utc` |
| `system_batt_dc_events` | `data/__tmp_batt_dc_events.txt000.gz` | Gzipped CSV | Pre-aggregated, needs column verification |
| `system_display_devices` | `data/display_devices.txt000.gz` | Gzipped CSV | Pre-aggregated from update |
| `system_userwait` | `data/userwait_v2/0000_part_00.parquet` | Parquet (1/16) | Needs aggregation: GROUP BY guid/proc_name_current/event_name/ac_dc_event_name |
| `system_frgnd_apps_types` | `data/__tmp_fgnd_apps_date.txt003.gz` | Gzipped CSV | Use `ignore_errors=true`; partial sample (1/4 files) |

---
## Query 1: `avg_platform_power_c0_freq_temp_by_chassis`
Type: Aggregate stats + 5-way join

SQL summary: Average power, C0 residency, frequency, and temperature by chassis type. Joins `sysinfo` with 4 hw_metric_stats-derived tables on `guid`.

Reporting tables: `system_sysinfo_unique_normalized`, `system_psys_rap_watts`, `system_pkg_C0`, `system_pkg_avg_freq_mhz`, `system_pkg_temp_centigrade`

In [12]:
hw_glob = str(DATA / "hw_metric_stats" / "*.parquet")
sysinfo_glob = str(DATA / "system_sysinfo_unique_normalized" / "*.parquet")

display(Markdown("sysinfo: need `chassistype`, `guid`"))
check_columns(sysinfo_glob, ["chassistype", "guid"])

display(Markdown("hw_metric_stats: need `guid`, `nrs`, `mean`, `name` (for filtering)"))
check_columns(hw_glob, ["guid", "nrs", "mean", "name"])

display(Markdown("Metric name availability:"))
lines = []
for metric, pattern in [
    ("system_psys_rap_watts", "RAP"),
    ("system_pkg_C0", "C0_RESIDENCY"),
    ("system_pkg_avg_freq_mhz", "AVG_FREQ"),
    ("system_pkg_temp_centigrade", "TEMPERATURE"),
]:
    r = con.execute(f"""
        SELECT name, COUNT(*) as n, COUNT(DISTINCT guid) as g
        FROM read_parquet('{hw_glob}')
        WHERE name LIKE '%{pattern}%'
        GROUP BY name
    """).df()
    if len(r) > 0:
        lines.append(f"- ✓ `{metric}`: {r['name'].tolist()} ({r['g'].sum()} guids)")
    else:
        lines.append(f"- ✗ `{metric}`: no rows matching `{pattern}`")
display(Markdown("\n".join(lines)))

sysinfo: need `chassistype`, `guid`

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `chassistype`
- ✓ `guid`

hw_metric_stats: need `guid`, `nrs`, `mean`, `name` (for filtering)

Rows: 56,196,901 | Guids: 28,896 | Columns: 16
- ✓ `guid`
- ✓ `nrs`
- ✓ `mean`
- ✓ `name`

Metric name availability:

- ✓ `system_psys_rap_watts`: ['HW::PACKAGE:RAP:WATTS:', 'HW:::PSYS_RAP:WATTS:'] (816 guids)
- ✓ `system_pkg_C0`: ['HW::PACKAGE:ANY_GFXE_C0:RESIDENCY:', 'HW:::GT_C0_RESIDENCY::', 'HW::PACKAGE:C0_RESIDENCY:PERCENT:'] (9722 guids)
- ✓ `system_pkg_avg_freq_mhz`: ['HW::CORE:AVG_FREQ:MHZ:'] (613 guids)
- ✓ `system_pkg_temp_centigrade`: ['HW::CORE:TEMPERATURE:CENTIGRADE:', 'HW:::CHIPSET_TEMPERATURE:CENTIGRADE:'] (1063 guids)

---
## Query 2: `server_exploration_1`
Type: Aggregate stats + join

SQL summary: Identify servers by network pattern (sent > received). Joins `system_network_consumption` with `sysinfo`.

Reporting tables: `system_sysinfo_unique_normalized`, `system_network_consumption`

Column mapping: Raw `input_description` → query uses `input_desc`. Raw `nr_samples` → query uses `nrs`. Raw `avg_bytes_sec` → same name.

In [13]:
net_glob = str(DATA / "os_network_consumption_v2" / "*.parquet")

display(Markdown("os_network_consumption_v2: query uses `guid`, `nrs`, `avg_bytes_sec`, `input_desc`\n\nRaw columns map: `input_description` to `input_desc`, `nr_samples` to `nrs`"))
check_columns(net_glob, ["guid", "nr_samples", "avg_bytes_sec", "input_description"])

display(Markdown("sysinfo: need `guid`, `chassistype`, `modelvendor_normalized`, `model_normalized`, `ram`, `os`, `#ofcores`"))
check_columns(sysinfo_glob, ["guid", "chassistype", "modelvendor_normalized", "model_normalized", "ram", "os", "#ofcores"])

os_network_consumption_v2: query uses `guid`, `nrs`, `avg_bytes_sec`, `input_desc`

Raw columns map: `input_description` to `input_desc`, `nr_samples` to `nrs`

Rows: 121,843,286 | Guids: 37,224 | Columns: 17
- ✓ `guid`
- ✓ `nr_samples`
- ✓ `avg_bytes_sec`
- ✓ `input_description`

sysinfo: need `guid`, `chassistype`, `modelvendor_normalized`, `model_normalized`, `ram`, `os`, `#ofcores`

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `chassistype`
- ✓ `modelvendor_normalized`
- ✓ `model_normalized`
- ✓ `ram`
- ✓ `os`
- ✓ `#ofcores`

---
## Query 3: `Xeon_network_consumption`
Type: Geographic/demographic

SQL summary: Compare Xeon vs non-Xeon network consumption by OS. Uses `cpuname` from sysinfo to classify.

Reporting tables: `system_sysinfo_unique_normalized`, `system_network_consumption`

In [14]:
display(Markdown("sysinfo: need `guid`, `cpuname`, `cpucode`, `os`"))
check_columns(sysinfo_glob, ["guid", "cpuname", "cpucode", "os"])

display(Markdown("os_network_consumption_v2: same as query 2"))
check_columns(net_glob, ["guid", "nr_samples", "avg_bytes_sec", "input_description"])

sysinfo: need `guid`, `cpuname`, `cpucode`, `os`

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `cpuname`
- ✓ `cpucode`
- ✓ `os`

os_network_consumption_v2: same as query 2

Rows: 121,843,286 | Guids: 37,224 | Columns: 17
- ✓ `guid`
- ✓ `nr_samples`
- ✓ `avg_bytes_sec`
- ✓ `input_description`

---
## Query 4: `most_popular_browser_in_each_country_by_system_count`
Type: Ranked top-k

SQL summary: Most popular browser per country by system count. Window function with `RANK()`.

Reporting tables: `system_sysinfo_unique_normalized`, `system_web_cat_usage`

In [15]:
web_glob = str(DATA / "web_cat_usage_v2" / "*.parquet")

display(Markdown("sysinfo — need: guid, countryname_normalized"))
check_columns(sysinfo_glob, ["guid", "countryname_normalized"])

display(Markdown("\nweb_cat_usage_v2 — need: guid, browser"))
check_columns(web_glob, ["guid", "browser"])

sysinfo — need: guid, countryname_normalized

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `countryname_normalized`


web_cat_usage_v2 — need: guid, browser

Rows: 21,354,922 | Guids: 64,276 | Columns: 18
- ✓ `guid`
- ✓ `browser`

---
## Query 5: `popular_browsers_by_count_usage_percentage`
Type: Histogram/distribution

SQL summary: Browser popularity stats , % of systems, % of instances, % of duration.

Reporting tables: `system_web_cat_usage`

In [16]:
display(Markdown("web_cat_usage_v2 — need: guid, browser, duration_ms"))
check_columns(web_glob, ["guid", "browser", "duration_ms"])

web_cat_usage_v2 — need: guid, browser, duration_ms

Rows: 21,354,922 | Guids: 64,276 | Columns: 18
- ✓ `guid`
- ✓ `browser`
- ✓ `duration_ms`

---
## Query 6: `persona_web_cat_usage_analysis`
Type: Complex multi-way pivot

SQL summary: Web category duration % by persona across 28 browsing categories. Joins `sysinfo` with `web_cat_pivot`.

Reporting tables: `system_sysinfo_unique_normalized`, `system_web_cat_pivot_duration`

Note: The query SQL uses shortened column names (e.g., `education`, `finance`, `mail`, `news`, `private`, `reference`, `search`, `shopping`, `unclassified`). Our pivot table uses full names (e.g., `education_education`, `finance_banking_and_accounting`). Need column aliasing in the reporting table build step.

In [17]:
pivot_glob = str(DATA / "web_cat_pivot" / "*.parquet")

display(Markdown("sysinfo — need: guid, persona"))
check_columns(sysinfo_glob, ["guid", "persona"])

display(Markdown("\nweb_cat_pivot — need: guid + 28 category columns"))
full_names = [
    "guid", "content_creation_photo_edit_creation", "content_creation_video_audio_edit_creation",
    "content_creation_web_design_development", "education_education",
    "entertainment_music_audio_streaming", "entertainment_other", "entertainment_video_streaming",
    "finance_banking_and_accounting", "games_other", "games_video_games", "mail_mail",
    "news_news", "other_unclassified", "private_private", "productivity_crm",
    "productivity_other", "productivity_presentations", "productivity_programming",
    "productivity_project_management", "productivity_spreadsheets", "productivity_word_processing",
    "recreation_travel", "reference_reference", "search_search", "shopping_shopping",
    "social_social_network", "social_communication", "social_communication_live",
]
check_columns(pivot_glob, full_names)

display(Markdown("\n⚠ Query SQL uses short aliases (education, finance, mail, etc.)."))
display(Markdown("  Reporting table must rename: education_education → education, etc."))


sysinfo — need: guid, persona

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `persona`


web_cat_pivot — need: guid + 28 category columns

Rows: 512,077 | Guids: 512,077 | Columns: 29
- ✓ `guid`
- ✓ `content_creation_photo_edit_creation`
- ✓ `content_creation_video_audio_edit_creation`
- ✓ `content_creation_web_design_development`
- ✓ `education_education`
- ✓ `entertainment_music_audio_streaming`
- ✓ `entertainment_other`
- ✓ `entertainment_video_streaming`
- ✓ `finance_banking_and_accounting`
- ✓ `games_other`
- ✓ `games_video_games`
- ✓ `mail_mail`
- ✓ `news_news`
- ✓ `other_unclassified`
- ✓ `private_private`
- ✓ `productivity_crm`
- ✓ `productivity_other`
- ✓ `productivity_presentations`
- ✓ `productivity_programming`
- ✓ `productivity_project_management`
- ✓ `productivity_spreadsheets`
- ✓ `productivity_word_processing`
- ✓ `recreation_travel`
- ✓ `reference_reference`
- ✓ `search_search`
- ✓ `shopping_shopping`
- ✓ `social_social_network`
- ✓ `social_communication`
- ✓ `social_communication_live`


⚠ Query SQL uses short aliases (education, finance, mail, etc.).

  Reporting table must rename: education_education → education, etc.

---
## Query 7: `ram_utilization_histogram`
Type: Histogram/distribution

SQL summary: RAM utilization histogram by memory capacity. Uses `sysinfo_ram` (needs JOIN), `avg_percentage_used`, `nrs`.

Reporting tables: `system_memory_utilization`

Column mapping (from Intel's ETL SQL):
- `average` = free memory in MB (NOT a percentage , confirmed by DCA dictionary and Intel's ETL)
- `sysinfo_ram = ram * 2^10` (sysinfo `ram` in GB → MB)
- `avg_free_ram = SUM(sample_count * average) / SUM(sample_count)` (weighted avg free MB)
- `avg_percentage_used = ROUND((sysinfo_ram - avg_free_ram) * 100 / sysinfo_ram)`
- `nrs = SUM(sample_count)`, grouped by `guid, dt`
- Must JOIN with sysinfo on guid, filter `ram != 0`

In [18]:
mem_glob = str(DATA / "os_memsam_avail_percent" / "*.parquet")

display(Markdown("os_memsam_avail_percent — need: guid, average (→ avg_percentage_used), sample_count (→ nrs)"))
check_columns(mem_glob, ["guid", "average", "sample_count"])

display(Markdown("\nsysinfo — need: guid, ram (→ sysinfo_ram)"))
check_columns(sysinfo_glob, ["guid", "ram"])

os_memsam_avail_percent — need: guid, average (→ avg_percentage_used), sample_count (→ nrs)

Rows: 21,688,089 | Guids: 69,552 | Columns: 34
- ✓ `guid`
- ✓ `average`
- ✓ `sample_count`


sysinfo — need: guid, ram (→ sysinfo_ram)

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `ram`

---
## Query 8: `pkg_power_by_country`
Type: Geographic/demographic

SQL summary: Average CPU package power by country. Joins `sysinfo` with `system_hw_pkg_power`.

Reporting tables: `system_sysinfo_unique_normalized`, `system_hw_pkg_power`

Data source: `hw_metric_stats` filtered to power metrics, using `nrs` and `mean` columns.

In [19]:
display(Markdown("sysinfo — need: guid, countryname_normalized"))
check_columns(sysinfo_glob, ["guid", "countryname_normalized"])

display(Markdown("\nhw_metric_stats (filtered to POWER) — need: guid, nrs, mean"))
check_columns(hw_glob, ["guid", "nrs", "mean"])

r = con.execute(f"""
    SELECT COUNT(*) as n, COUNT(DISTINCT guid) as g
    FROM read_parquet('{hw_glob}')
    WHERE name LIKE '%IA_POWER%'
""").fetchone()
display(Markdown(f"\n  IA_POWER rows: {r[0]:,} across {r[1]:,} guids"))


sysinfo — need: guid, countryname_normalized

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `countryname_normalized`


hw_metric_stats (filtered to POWER) — need: guid, nrs, mean

Rows: 56,196,901 | Guids: 28,896 | Columns: 16
- ✓ `guid`
- ✓ `nrs`
- ✓ `mean`


  IA_POWER rows: 83,775 across 800 guids

---
## Query 9: `battery_power_on_geographic_summary`
Type: Geographic breakdown

SQL summary: Battery usage by country , avg power ons, avg duration. Joins `batt_dc_events` with `sysinfo`.

Reporting tables: `system_batt_dc_events`, `system_sysinfo_unique_normalized`

Note: Raw `__tmp_batt_dc_events` is event-level (one row per DC power-on event). The reporting schema (per Intel's definition) is per `(guid, dt)`. Must aggregate: `GROUP BY guid, dt → COUNT(*) AS num_power_ons, SUM(duration_mins) AS duration_mins` (plus battery percentage stats).

In [20]:
batt_file = str(DATA / "__tmp_batt_dc_events.txt000.gz")

display(Markdown("__tmp_batt_dc_events — raw event-level data, needs aggregation"))
display(Markdown("  Raw columns: guid, power_on_dc_ts, duration_mins, power_on/off_battery_percent"))
display(Markdown("  Reporting table needs: guid, num_power_ons (= COUNT(*)), duration_mins (= SUM)"))
check_columns(batt_file, ["guid", "duration_mins", "power_on_dc_ts"])

display(Markdown("\nSample aggregated output (reporting table preview):"))
preview = con.execute(f"""
    SELECT guid, COUNT(*) as num_power_ons, SUM(duration_mins) as duration_mins
    FROM read_csv('{batt_file}', auto_detect=true)
    GROUP BY guid
    ORDER BY num_power_ons DESC
    LIMIT 5
""").df()
display(preview)

display(Markdown("\nsysinfo — need: guid, countryname_normalized"))
check_columns(sysinfo_glob, ["guid", "countryname_normalized"])


__tmp_batt_dc_events — raw event-level data, needs aggregation

  Raw columns: guid, power_on_dc_ts, duration_mins, power_on/off_battery_percent

  Reporting table needs: guid, num_power_ons (= COUNT(*)), duration_mins (= SUM)

Rows: 939,020 | Guids: 19,780 | Columns: 6
- ✓ `guid`
- ✓ `duration_mins`
- ✓ `power_on_dc_ts`


Sample aggregated output (reporting table preview):

,guid,num_power_ons,duration_mins
0,4d70bce0a5e24588bf05d3569a10e3e4,2548,17959.0
1,bffab3e295f24807ab997854fc8fb574,1085,17087.0
2,041534d5c974485786106859258cffda,983,62615.0
3,5b71e72ac0ed42599045f2882f53d505,915,26782.0
4,3b3ba8ef309a439782948464f92ad109,878,46944.0



sysinfo — need: guid, countryname_normalized

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 32
- ✓ `guid`
- ✓ `countryname_normalized`

---
## Query 10: `battery_on_duration_cpu_family_gen`
Type: Demographic breakdown

SQL summary: Battery duration by CPU family/generation. Joins `cpu_metadata` with `batt_dc_events`.

Reporting tables: `system_cpu_metadata`, `system_batt_dc_events`

In [21]:
cpu_file = str(DATA / "system_cpu_metadata.txt000.gz")

display(Markdown("system_cpu_metadata — need: guid, marketcodename, cpugen"))
check_columns(cpu_file, ["guid", "marketcodename", "cpugen"])

display(Markdown("\n__tmp_batt_dc_events — need: guid, duration_mins (aggregated per guid)"))
check_columns(batt_file, ["guid", "duration_mins"])

system_cpu_metadata — need: guid, marketcodename, cpugen

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 12
- ✓ `guid`
- ✓ `marketcodename`
- ✓ `cpugen`


__tmp_batt_dc_events — need: guid, duration_mins (aggregated per guid)

Rows: 939,020 | Guids: 19,780 | Columns: 6
- ✓ `guid`
- ✓ `duration_mins`

---
## Query 11: `on_off_mods_sleep_summary_by_cpu_marketcodename_gen`
Type: Demographic breakdown

SQL summary: On/off/mods/sleep time stats by CPU generation. Joins `on_off_suspend_time_day` with `cpu_metadata`.

Reporting tables: `system_on_off_suspend_time_day`, `system_cpu_metadata`

In [22]:
onoff_file = str(DATA / "guids_on_off_suspend_time_day.txt000.gz")

display(Markdown("guids_on_off_suspend_time_day — need: guid, on_time, off_time, mods_time, sleep_time"))
check_columns(onoff_file, ["guid", "on_time", "off_time", "mods_time", "sleep_time"])

display(Markdown("\nsystem_cpu_metadata — need: guid, marketcodename, cpugen"))
check_columns(cpu_file, ["guid", "marketcodename", "cpugen"])

guids_on_off_suspend_time_day — need: guid, on_time, off_time, mods_time, sleep_time

Rows: 1,582,017 | Guids: 36,958 | Columns: 7
- ✓ `guid`
- ✓ `on_time`
- ✓ `off_time`
- ✓ `mods_time`
- ✓ `sleep_time`


system_cpu_metadata — need: guid, marketcodename, cpugen

Rows: 1,000,000 | Guids: 1,000,000 | Columns: 12
- ✓ `guid`
- ✓ `marketcodename`
- ✓ `cpugen`

---
## Query 12: `mods_blockers_by_osname_and_codename`
Type: Aggregate stats + join

SQL summary: Count distribution of modern standby blockers by OS name/codename. Joins `mods_top_blocker_hist` with `os_codename_history` on guid + date range.

Reporting tables: `system_mods_top_blocker_hist`, `system_os_codename_history`

Note: The blocker table uses `dt_utc` as date; query references `dt`. Need alias.

In [23]:
blocker_file = str(DATA / "mods_sleepstudy_top_blocker_hist.txt000.gz")
codename_file = str(DATA / "system_os_codename_history.txt000.gz")

display(Markdown("mods_sleepstudy_top_blocker_hist — need: guid, dt (or dt_utc)"))
check_columns(blocker_file, ["guid", "dt_utc"])

display(Markdown("\nsystem_os_codename_history — need: guid, min_ts, max_ts, os_name, os_codename"))
check_columns(codename_file, ["guid", "min_ts", "max_ts", "os_name", "os_codename"])

mods_sleepstudy_top_blocker_hist — need: guid, dt (or dt_utc)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 92,460,980 | Guids: 65,034 | Columns: 13
- ✓ `guid`
- ✓ `dt_utc`


system_os_codename_history — need: guid, min_ts, max_ts, os_name, os_codename

Rows: 639,223 | Guids: 299,099 | Columns: 6
- ✓ `guid`
- ✓ `min_ts`
- ✓ `max_ts`
- ✓ `os_name`
- ✓ `os_codename`

---
## Query 13: `top_mods_blocker_types_durations_by_osname_and_codename`
Type: Aggregate stats + join

SQL summary: Blocker name/type/activity with duration stats by OS. Same join as query 12 plus blocker detail columns.

Reporting tables: `system_mods_top_blocker_hist`, `system_os_codename_history`

In [24]:
display(Markdown("mods_sleepstudy_top_blocker_hist — need: guid, dt_utc, blocker_name, active_time_ms, activity_level, blocker_type"))
check_columns(blocker_file, ["guid", "dt_utc", "blocker_name", "active_time_ms", "activity_level", "blocker_type"])

display(Markdown("\nsystem_os_codename_history — same as query 12"))
check_columns(codename_file, ["guid", "min_ts", "max_ts", "os_name", "os_codename"])

mods_sleepstudy_top_blocker_hist — need: guid, dt_utc, blocker_name, active_time_ms, activity_level, blocker_type

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 92,460,980 | Guids: 65,034 | Columns: 13
- ✓ `guid`
- ✓ `dt_utc`
- ✓ `blocker_name`
- ✓ `active_time_ms`
- ✓ `activity_level`
- ✓ `blocker_type`


system_os_codename_history — same as query 12

Rows: 639,223 | Guids: 299,099 | Columns: 6
- ✓ `guid`
- ✓ `min_ts`
- ✓ `max_ts`
- ✓ `os_name`
- ✓ `os_codename`

---
## Query 14: `display_devices_connection_type_resolution_durations_ac_dc`
Type: Aggregate stats + join

SQL summary: Display connection types and resolutions with AC/DC duration stats. Single-table query on `system_display_devices`.

Reporting tables: `system_display_devices`

In [25]:
display_file = str(DATA / "display_devices.txt000.gz")

display(Markdown("display_devices — need: guid, connection_type, resolution_heigth, resolution_width, duration_ac, duration_dc"))
check_columns(display_file, ["guid", "connection_type", "resolution_heigth", "resolution_width", "duration_ac", "duration_dc"])

display_devices — need: guid, connection_type, resolution_heigth, resolution_width, duration_ac, duration_dc

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 220,997,262 | Guids: 209,239 | Columns: 23
- ✓ `guid`
- ✓ `connection_type`
- ✓ `resolution_heigth`
- ✓ `resolution_width`
- ✓ `duration_ac`
- ✓ `duration_dc`

---
## Query 15: `display_devices_vendors_percentage`
Type: Aggregate stats

SQL summary: Display vendor market share by system count percentage. Single-table query on `system_display_devices`.

Reporting tables: `system_display_devices`

In [26]:
display(Markdown("display_devices — need: guid, vendor_name"))
check_columns(display_file, ["guid", "vendor_name"])

display_devices — need: guid, vendor_name

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 220,997,262 | Guids: 209,239 | Columns: 23
- ✓ `guid`
- ✓ `vendor_name`

---
## Query 16: `userwait_top_10_wait_processes`
Type: Ranked top-k

SQL summary: Top 10 worst apps by average wait time. Needs aggregated `total_duration_ms` and `number_of_instances`.

Reporting tables: `system_userwait`

Column mapping: `proc_name_current` → `proc_name`, `ac_dc_event_name` → `acdc`, `SUM(duration_ms)` → `total_duration_ms`, `COUNT(*)` → `number_of_instances`

In [27]:
uw_file = str(DATA / "userwait_v2" / "0000_part_00.parquet")

display(Markdown("userwait_v2 (raw) — need: guid, proc_name_current (→ proc_name), event_name, duration_ms (→ SUM), ac_dc_event_name (→ acdc)"))
check_columns(uw_file, ["guid", "proc_name_current", "event_name", "duration_ms", "ac_dc_event_name"])

display(Markdown("\nSample aggregated output (reporting table preview):"))
preview = con.execute(f"""
    SELECT 
        UPPER(proc_name_current) AS proc_name,
        event_name,
        ac_dc_event_name AS acdc,
        SUM(duration_ms) AS total_duration_ms,
        COUNT(*) AS number_of_instances
    FROM read_parquet('{uw_file}')
    GROUP BY proc_name_current, event_name, ac_dc_event_name
    ORDER BY total_duration_ms DESC
    LIMIT 5
""").df()
display(preview)


userwait_v2 (raw) — need: guid, proc_name_current (→ proc_name), event_name, duration_ms (→ SUM), ac_dc_event_name (→ acdc)

Rows: 175,223,880 | Guids: 38,142 | Columns: 28
- ✓ `guid`
- ✓ `proc_name_current`
- ✓ `event_name`
- ✓ `duration_ms`
- ✓ `ac_dc_event_name`


Sample aggregated output (reporting table preview):

,proc_name,event_name,acdc,total_duration_ms,number_of_instances
0,NA,TOTAL_NON_WAIT_EVENTS,NA,5.458816e+13,2712812
1,DUMMY_PROCESS,APPSTARTING,UN_DISPLAY_UN,1.291225e+11,7613997
2,DUMMY_PROCESS,APPSTARTING,unknown,1.114562e+11,5596373
3,DUMMY_PROCESS,APPSTARTING,AC_DISPLAY_ON,9.602288e+10,2685459
4,CHROME.EXE,APPSTARTING,UN_DISPLAY_UN,5.880026e+10,5341088


---
## Query 17: `userwait_top_10_wait_processes_wait_type_ac_dc`
Type: Ranked top-k + pivot by power state

SQL summary: Top 10 wait apps split by event type (APPSTARTING/WAIT) and power state (AC/DC). Same base table as query 16.

Reporting tables: `system_userwait`

In [28]:
display(Markdown("userwait_v2 (raw) — same columns as query 16"))
check_columns(uw_file, ["guid", "proc_name_current", "event_name", "duration_ms", "ac_dc_event_name"])

display(Markdown("\nevent_name values:"))
display(con.execute(f"SELECT event_name, COUNT(*) n FROM read_parquet('{uw_file}') GROUP BY 1 ORDER BY n DESC").df())

display(Markdown("\nac_dc_event_name values:"))
display(con.execute(f"SELECT ac_dc_event_name, COUNT(*) n FROM read_parquet('{uw_file}') GROUP BY 1 ORDER BY n DESC").df())


userwait_v2 (raw) — same columns as query 16

Rows: 175,223,880 | Guids: 38,142 | Columns: 28
- ✓ `guid`
- ✓ `proc_name_current`
- ✓ `event_name`
- ✓ `duration_ms`
- ✓ `ac_dc_event_name`


event_name values:

,event_name,n
0,APPSTARTING,142814397
1,WAIT,26983605
2,TOTAL_DISCARDED_WAIT_EVENTS,2713066
3,TOTAL_NON_WAIT_EVENTS,2712812



ac_dc_event_name values:

,ac_dc_event_name,n
0,UN_DISPLAY_UN,44958207
1,AC_DISPLAY_OFF,44059521
2,unknown,42808890
3,AC_DISPLAY_ON,16077082
4,DC_DISPLAY_ON,6841200
5,DC_DISPLAY_OFF,5829682
6,NA,5425878
7,UN_DISPLAY_OFF,5155463
8,DC_DISPLAY_UN,1822000
9,UN_DISPLAY_ON,1517243


---
## Query 18: `userwait_top_20_wait_processes_compare_ac_dc_unknown_durations`
Type: Ranked top-k + AC/DC/UN pivot

SQL summary: Top 20 wait apps with duration pivoted by power state (AC, DC, Unknown). Same base table as queries 16-17.

Reporting tables: `system_userwait`

In [29]:
display(Markdown("userwait_v2 (raw) — same columns as queries 16-17, already verified above"))
check_columns(uw_file, ["guid", "proc_name_current", "event_name", "duration_ms", "ac_dc_event_name"])

userwait_v2 (raw) — same columns as queries 16-17, already verified above

Rows: 175,223,880 | Guids: 38,142 | Columns: 28
- ✓ `guid`
- ✓ `proc_name_current`
- ✓ `event_name`
- ✓ `duration_ms`
- ✓ `ac_dc_event_name`

---
## Query 19: `top_10_applications_by_app_type_ranked_by_focal_time`
Type: Ranked top-k

SQL summary: Top 10 apps per app type by average focal screen time per day.

Reporting tables: `system_frgnd_apps_types`

Note: Query references `totalsecfocal_day` (mapped from `totalsecfocal_day` in our data , exact match).

In [30]:
fgnd_file = str(DATA / "__tmp_fgnd_apps_date.txt003.gz")

display(Markdown("__tmp_fgnd_apps_date — need: guid, app_type, exe_name, totalsecfocal_day"))
check_columns(fgnd_file, ["guid", "app_type", "exe_name", "totalsecfocal_day"], ignore_errors=True)

display(Markdown("\napp_type distribution:"))
r = con.execute(f"""
    SELECT app_type, COUNT(*) n, COUNT(DISTINCT guid) g
    FROM read_csv('{fgnd_file}', auto_detect=true, ignore_errors=true)
    GROUP BY 1 ORDER BY n DESC LIMIT 10
""").df()
display(r)


__tmp_fgnd_apps_date — need: guid, app_type, exe_name, totalsecfocal_day

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 56,755,998 | Guids: 55,830 | Columns: 9
- ✓ `guid`
- ✓ `app_type`
- ✓ `exe_name`
- ✓ `totalsecfocal_day`


app_type distribution:

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,app_type,n,g
0,Other,20156630,55646
1,Web Browsing,7120698,55385
2,System,6846906,55206
3,Productivity,5734317,50607
4,Utilities + tools,3598338,53875
5,Gaming,2516002,28910
6,Communication,2244716,30112
7,Social,1820283,29803
8,Gaming auxiliary,1781299,25448
9,Security,1352893,32384


---
## Query 20: `top_10_applications_by_app_type_ranked_by_system_count`
Type: Ranked top-k

SQL summary: Top 10 apps per app type by number of distinct clients. Same base table as query 19.

Reporting tables: `system_frgnd_apps_types`

In [31]:
display(Markdown("__tmp_fgnd_apps_date — need: guid, app_type, exe_name"))
check_columns(fgnd_file, ["guid", "app_type", "exe_name"], ignore_errors=True)

__tmp_fgnd_apps_date — need: guid, app_type, exe_name

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 56,755,998 | Guids: 55,830 | Columns: 9
- ✓ `guid`
- ✓ `app_type`
- ✓ `exe_name`

---
## Query 21: `top_10_applications_by_app_type_ranked_by_total_detections`
Type: Ranked top-k

SQL summary: Top 10 apps per app type by total detections (`lines_per_day`). Same base table as queries 19-20.

Reporting tables: `system_frgnd_apps_types`

In [32]:
display(Markdown("__tmp_fgnd_apps_date — need: guid, app_type, exe_name, lines_per_day"))
check_columns(fgnd_file, ["guid", "app_type", "exe_name", "lines_per_day"], ignore_errors=True)

__tmp_fgnd_apps_date — need: guid, app_type, exe_name, lines_per_day

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Rows: 56,755,998 | Guids: 55,830 | Columns: 9
- ✓ `guid`
- ✓ `app_type`
- ✓ `exe_name`
- ✓ `lines_per_day`

---
## Queries 22-24: Power consumption rankings (single-guid stub data)

Queries:
- `ranked_process_classifications` , rank `user_id` by total power consumption
- `top_10_processes_per_user_id_ranked_by_total_power_consumption` , top 10 apps per `user_id`
- `top_20_most_power_consuming_processes_by_avg_power_consumed` , top 20 apps overall

Type: Ranked top-k

Reporting table: `system_mods_power_consumption`

Data source: `mods_sleepstudy_power_estimation_data_13wks.txt000.gz` (218 KB, 10K rows, 1 guid only)

Why this is the only source: The DCA data dictionary (144 pages, 115 ETL tables) was exhaustively searched. Per-process power consumption with `user_id` classification comes exclusively from Windows Modern Standby sleep study reports (`powercfg /sleepstudy /xml`). No other table in the DCA corpus has these columns:
- `plist_process_resource_util` has `proc_name` but no `user_id` or power watts (only CPU sec, IO bytes, memory)
- `frgnd_system_usage_by_app` might have per-app power metrics but is 337.5 GiB and also lacks `user_id`
- `mods_sleepstudy_scenario_instance` has session-level key-value pairs, not per-process power
- `mods_sleepstudy_top_blocker` has blocker data, not power consumption

Why it still works: None of the 3 queries reference `guid`. They aggregate purely by `user_id` (5 distinct values) and `app_id` (100+ distinct processes). The ranking structure is rich enough for DP benchmark comparison , synthesize from this table, run queries on both, compare rankings.

In [33]:
power_file = str(DATA / "mods_sleepstudy_power_estimation_data_13wks.txt000.gz")

display(Markdown("mods_sleepstudy_power_estimation_data_13wks — need: user_id, app_id, total_power_consumption"))
check_columns(power_file, ["guid", "user_id", "app_id", "total_power_consumption"])

display(Markdown("\n=== Data profile ==="))
profile = con.execute(f"""
    SELECT 
        COUNT(*) as total_rows,
        COUNT(DISTINCT guid) as n_guids,
        COUNT(DISTINCT user_id) as n_user_ids,
        COUNT(DISTINCT app_id) as n_app_ids,
        MIN(total_power_consumption) as min_power,
        ROUND(AVG(total_power_consumption), 1) as avg_power,
        MAX(total_power_consumption) as max_power
    FROM read_csv('{power_file}', auto_detect=true)
""").df()
display(Markdown(profile.to_string(index=False)))


display(Markdown("\n=== user_id distribution (query 22 groups by this) ==="))
uid = con.execute(f"""
    SELECT user_id, COUNT(*) as n, SUM(total_power_consumption) as total_power
    FROM read_csv('{power_file}', auto_detect=true)
    GROUP BY user_id ORDER BY total_power DESC
""").df()
display(uid)

display(Markdown("\n=== Top 10 app_id by power (query 24 ranks these) ==="))
apps = con.execute(f"""
    SELECT app_id, AVG(total_power_consumption) as avg_power, COUNT(*) as n
    FROM read_csv('{power_file}', auto_detect=true)
    GROUP BY app_id ORDER BY avg_power DESC LIMIT 10
""").df()
display(apps)

display(Markdown("\n⚠ Single-guid limitation: 10K rows from 1 client's sleep study power profile."))
display(Markdown("  Queries work because they GROUP BY user_id/app_id, not guid."))
display(Markdown("  For DP benchmark: synthesize → run same queries → compare rankings."))



mods_sleepstudy_power_estimation_data_13wks — need: user_id, app_id, total_power_consumption

Rows: 10,000 | Guids: 1 | Columns: 21
- ✓ `guid`
- ✓ `user_id`
- ✓ `app_id`
- ✓ `total_power_consumption`


=== Data profile ===

 total_rows  n_guids  n_user_ids  n_app_ids  min_power  avg_power  max_power
      10000        1           5        549          0       27.8       6767


=== user_id distribution (query 22 groups by this) ===

...


=== Top 10 app_id by power (query 24 ranks these) ===

,app_id,avg_power,n
0,\Device\HarddiskVolume3\Program Files\Google\C...,2526.740741,27
1,\Device\HarddiskVolume3\Program Files (x86)\Mi...,2287.928571,28
2,Unknown,1496.105263,19
3,4DF9E0F8.Netflix_6.99.5.0_x64__mcm4njqhnhss8,1343.900000,10
4,\Device\HarddiskVolume3\Program Files\Microsof...,517.500000,2
5,System,366.452381,42
6,\Device\HarddiskVolume3\Program Files\Microsof...,299.307692,13
7,\Device\HarddiskVolume3\Windows\explorer.exe,290.138889,36
8,\Device\HarddiskVolume3\Program Files\HP\HP On...,220.666667,18
9,System Interrupts,210.904762,42



⚠ Single-guid limitation: 10K rows from 1 client's sleep study power profile.

  Queries work because they GROUP BY user_id/app_id, not guid.

  For DP benchmark: synthesize → run same queries → compare rankings.

---
## Summary (24/24 queries)

In [34]:
display(Markdown("All 24 benchmark queries verified."))
display(Markdown())

display(Markdown("  21 queries: multi-guid population data (thousands of clients)"))
display(Markdown("   3 queries: single-guid stub data (10K rows, power consumption rankings)"))
display(Markdown())

display(Markdown("Reporting tables that need aggregation/transformation:"))
display(Markdown("  1. system_userwait         ← userwait_v2 (GROUP BY guid/proc/event/acdc, SUM/COUNT)"))
display(Markdown("  2. system_psys_rap_watts   ← hw_metric_stats (filter name LIKE '%RAP%')"))
display(Markdown("  3. system_pkg_C0           ← hw_metric_stats (filter name LIKE '%C0_RESIDENCY%')"))
display(Markdown("  4. system_pkg_avg_freq_mhz ← hw_metric_stats (filter name LIKE '%AVG_FREQ%')"))
display(Markdown("  5. system_pkg_temp         ← hw_metric_stats (filter name LIKE '%TEMPERATURE%')"))
display(Markdown("  6. system_hw_pkg_power     ← hw_metric_stats (filter name LIKE '%IA_POWER%')"))
display(Markdown("  7. system_network_consumption ← os_network_consumption_v2 (rename input_description→input_desc, nr_samples→nrs)"))
display(Markdown("  8. system_memory_utilization  ← os_memsam_avail_percent (100-average, JOIN sysinfo for ram)"))
display(Markdown("  9. system_web_cat_pivot_duration ← web_cat_pivot (rename full→short category names)"))
display(Markdown("  10. system_mods_top_blocker_hist ← mods_sleepstudy_top_blocker_hist (alias dt_utc→dt)"))
display(Markdown("  11. system_batt_dc_events     ← __tmp_batt_dc_events (GROUP BY guid, COUNT→num_power_ons, SUM→duration_mins)"))
display(Markdown())

display(Markdown("Reporting tables usable directly (or with minimal renaming):"))
display(Markdown("  system_sysinfo_unique_normalized, system_web_cat_usage, system_cpu_metadata,"))
display(Markdown("  system_os_codename_history, system_on_off_suspend_time_day,"))
display(Markdown("  system_display_devices, system_frgnd_apps_types, system_mods_power_consumption"))
display(Markdown())

display(Markdown("Single-guid limitation (3 power queries):"))
display(Markdown("  system_mods_power_consumption ← mods_sleepstudy_power_estimation_data_13wks (10K rows, 1 guid)"))
display(Markdown("  Exhaustive search of all 115 DCA ETL tables confirms no alternative source exists."))
display(Markdown("  Queries aggregate by user_id/app_id only (no guid reference) — usable for DP benchmark."))



All 24 benchmark queries verified.

<IPython.core.display.Markdown object>

  21 queries: multi-guid population data (thousands of clients)

   3 queries: single-guid stub data (10K rows, power consumption rankings)

<IPython.core.display.Markdown object>

...

<IPython.core.display.Markdown object>

Single-guid limitation (3 power queries):

  system_mods_power_consumption ← mods_sleepstudy_power_estimation_data_13wks (10K rows, 1 guid)

  Exhaustive search of all 115 DCA ETL tables confirms no alternative source exists.

  Queries aggregate by user_id/app_id only (no guid reference) — usable for DP benchmark.